<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anytree

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [ ]:
import pickle

# Reading from a file using Pickle
with open(f"{base_path}conversations_readability.pkl", 'rb') as file:
    conversations = pickle.load(file)

# Print the loaded variables
print("Len conversations:", len(conversations))

Len conversations: 32990


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def contains_reasoning(text):
    doc = nlp(text)

    # Check for specific part-of-speech patterns indicating reasoning
    reasoning_patterns = ["because", "since", "therefore", "due to", "as a result", "consequently", "thus",
                          "for this reason", "in conclusion", "owing to", "on account of", "resulting in",
                          "so", "hence", "in light of", "accordingly", "on the grounds that"]

    for token in doc:
        if token.text.lower() in reasoning_patterns:
            return True

    return False

In [ ]:
VERBOSE = 3000

In [ ]:
 def calculate_reasoning_existance_for_conversations():
  counter = 0
  for id in conversations:
    counter += 1
    comments = conversations[id]['comments']
    for comment in comments:
      if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
          has_reasoning = contains_reasoning(comment['body'])
          comment['has_reasoning'] = has_reasoning
    if counter % VERBOSE == 0:
      print(counter, "/", len(conversations))

In [ ]:
import pickle

def write_conversations_with_reasoning():
  with open(f"{base_path}conversations_reasoning.pkl", 'wb') as file:
    pickle.dump(conversations, file)
  print("Variables written to the file.")

In [ ]:
def extract_comments(conversations, num_break_points):
  flat_comments_list = []
  break_points_index = []
  counter = 0
  break_length = int(len(conversations)/num_break_points)
  print("break len ", break_length)
  for id in conversations:
    if counter % break_length == 0:
      break_points_index.append(len(flat_comments_list))
    counter += 1
    comments = conversations[id]['comments']
    for comment in comments:
      flat_comments_list.append(comment)

  return flat_comments_list, break_points_index

In [ ]:
import pickle

def write_reasoning_dic(reasoning_dic, file_path):
  with open(file_path, 'wb') as file:
      pickle.dump(reasoning_dic, file)
  print("Variables written to the file.")

In [39]:
import pickle

def merge_conversations_with_reasoning(conversations, file_pathes):
  reasoning_dic = dict()
  for file_path in file_pathes:
    with open(file_path, 'rb') as file:
      reasoning_part = pickle.load(file)
      reasoning_dic.update(reasoning_part)

  for id in conversations:
    comments = conversations[id]['comments']
    for comment in comments:
      if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
        comment['has_reasoning'] = reasoning_dic[comment['id']]
  with open(f"{base_path}conversations_with_reasoning.pkl", 'wb') as file:
    pickle.dump(conversations, file)
  print("Variables written to the file.")

In [ ]:
def calculate_reasoning_for_corpus(corpus):
  reasoning_dic = dict()
  for comment in corpus:
      if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
          has_reasoning = contains_reasoning(comment['body'])
          reasoning_dic[comment['id']] = has_reasoning
  return reasoning_dic

In [ ]:
import os

def calculate_reasoning(conversations, num_of_parts, rewrite):
  flat_comments, break_points_index = extract_comments(conversations, num_of_parts)
  print("len ", len(break_points_index))
  break_points_index.append(len(flat_comments))
  file_pathes = []
  for i in range(1, len(break_points_index)):
    print("range ", break_points_index[i-1], break_points_index[i])
    file_path = f"{base_path}comments_reasoning_{break_points_index[i-1]}_{break_points_index[i]}.pkl"
    file_pathes.append(file_path)
    print("path ", file_path)
    if rewrite or not os.path.exists(file_path):
      print("do")
      corpus = flat_comments[break_points_index[i-1]:break_points_index[i]]
      reasoning_dic = calculate_reasoning_for_corpus(corpus)
      write_reasoning_dic(reasoning_dic, file_path)
  merge_conversations_with_reasoning(conversations, file_pathes)

In [40]:
calculate_reasoning(conversations, 10, False)

break len  3299
len  10
range  0 53506
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_0_53506.pkl
range  53506 116967
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_53506_116967.pkl
range  116967 178801
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_116967_178801.pkl
range  178801 234319
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_178801_234319.pkl
range  234319 300115
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_234319_300115.pkl
range  300115 357006
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_300115_357006.pkl
range  357006 424960
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_357006_424960.pkl
range  424960 478226
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_424960_478226.pkl
range  478226 533555
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_478226_533555.pkl
range  53

In [41]:
write_conversations_with_reasoning()

Variables written to the file.


In [ ]:
from transformers import pipeline

# This model is a `zero-shot-classification` model.
# It will classify text, except you are free to choose any label you might imagine
classifier = pipeline(model="facebook/bart-large-mnli")
classifier(
    "I have a problem with my iphone that needs to be resolved asap!!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

In [ ]:
from transformers import pipeline

def contains_reasoning_pipeline(text):
    # Load the BERT model for text classification
    classifier = pipeline('text-classification')

    # Define a prompt to classify
    prompt = f"Is the following text reasoning about something? {text}"

    # Use the BERT model to classify the prompt
    print(classifier(prompt))

    # Check if the predicted label is positive (indicating reasoning)
    return False


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9996144771575928}]
The text does not contain reasoning.


In [ ]:
from transformers import pipeline

question_answerer = pipeline(task="question-answering")
preds = question_answerer(
    question="What is the reason?",
    context="because of everything you said earlier.",
)
print(
    f"score: {round(preds['score'], 4)}, start: {preds['start']}, end: {preds['end']}, answer: {preds['answer']}"
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


score: 0.2928, start: 0, end: 38, answer: because of everything you said earlier
